In [0]:
from zipfile import ZipFile
import os
from google.colab import files
! pip install tensorboardx
! pip install bayesian-optimization

ROOT = '/content'

Install ngrok for tunneling

In [0]:
if not os.path.exists(os.path.join(ROOT, "ngrok-stable-linux-amd64.zip")):
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip
  
LOG_DIR = os.path.join(ROOT,'saved/log/')
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

get_ipython().system_raw('./ngrok http 6006 &')

Setup tensorboard

In [0]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

Upload project as .zip file and extract it

In [0]:
file = files.upload()

zip_file = ZipFile(os.path.join(ROOT,list(file.keys())[0]))
zip_file.extractall(ROOT)
zip_file.close()

Modify parameters and train model **manually**

In [0]:
from config import CONFIG  
from parse_config import ConfigParser
from train import main

CONFIG['arch']['args']['num_channels'] = 64
CONFIG['arch']['args']['num_stacks'] = 4
CONFIG['arch']['args']['num_blocks'] = 7
CONFIG['arch']['args']['kernel_size'] = 5
CONFIG['sigma'] = 20
CONFIG['prediction_blur'] = 20
CONFIG['threshold'] = 0.01
CONFIG['trainer']['epochs'] = 50

main(ConfigParser(CONFIG))

Do **automatic** Bayesian optimization

In [0]:
from bayes_opt_train import run_bayes_opt

pbounds = {
    'num_channels': (6, 8),  # {64, 128, 256}
    'num_stacks': (2, 8),
    'num_blocks': (1, 7),
    'kernel_size': (1, 4),  # {3, 5, 7, 9}
    # 'sigma': (1, 10),
    # 'prediction_blur': (1, 10),
    # 'threshold': (0.00001, 0.2)
}

run_bayes_opt(pbounds, init_points=10, n_iter=10)